# Conectando OpenAI con python a BBDD (págila): Extracción de metadatos

In [1]:
from utils import get_embedding, full_md_extraction_pipeline, simple_md_extraction_pipeline, search_most_similar_md, conect_to_bbdd, close_conection
from utils import store_metadata_df_to_pickle, enviar_promt_chat_completions_mode
from calcular_tokens import num_tokens_from_string, num_tokens_from_messages

import pandas as pd
import openai
import numpy as np
import os

In [2]:
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Acceder a la API key
api_key = os.getenv("API_KEY")
openai.api_key = api_key

In [3]:
# mi_prompt = [
#     {'role': 'system',    'content':'eres un asistente agradable y gracioso'},               
#     {'role': 'user',      'content':'hola que tal estas'},
#     #{'role': 'assistant', 'content': 'Devuélveme la respuesta en este formato: \n```sql\nEL CODIGO SQL;\n```'}
# ]

# respuesta = enviar_promt_chat_completions_mode(mensaje=mi_prompt)

# print(respuesta)

## Carga de metadatos de la BBDD

Si el archivo.pickle existe en el directorio con los metadatos nos dice que ya existe, de lo contrario, los extrae de la bbdd. LA BASE DE DATOS DEBE ESTAR LEVANTADA 

In [4]:
pagila = {
    'bbdd_name':'pagila', 
    'user'     :'postgres', 
    'password' :'123', 
    'host'     :'localhost', 
    'port'     :'5432'
}

store_metadata_df_to_pickle(**pagila)

El archivo ya existe: data/pagila_simple_metadata.pickle
El archivo ya existe: data/pagila_full_metadata.pickle


'Proceso de almacenamiento de metadatos para pagila completado.'

In [5]:
rna_central = {
    'bbdd_name': 'pfmegrnargs', 
    'user'     : 'reader', 
    'password' : 'NWDMCE5xdipIjRrp', 
    'host'     : 'hh-pgsql-public.ebi.ac.uk', 
    'port'     : '5432'
}

store_metadata_df_to_pickle(**rna_central)

El archivo ya existe: data/pfmegrnargs_simple_metadata.pickle
El archivo ya existe: data/pfmegrnargs_full_metadata.pickle


'Proceso de almacenamiento de metadatos para pfmegrnargs completado.'

In [6]:
simple_md_df = pd.read_pickle('data/pagila_simple_metadata.pickle')   # pfmegrnargs   # pagila
full_md_df = pd.read_pickle('data/pagila_full_metadata.pickle')

display(simple_md_df.head())
display(full_md_df.head())

,metadata_str,md_str_tokens,embedding
film,"{""film"": [""film_id"", ""title"", ""description"", ""...",53,"[-0.010325274430215359, 0.012428195215761662, ..."
language,"{""language"": [""language_id"", ""name"", ""last_upd...",14,"[-0.029717877507209778, 0.01876918599009514, 0..."
actor,"{""actor"": [""actor_id"", ""first_name"", ""last_nam...",19,"[-0.03288605436682701, -0.005693415179848671, ..."
store,"{""store"": [""store_id"", ""manager_staff_id"", ""ad...",20,"[-0.029268378391861916, 0.008447147905826569, ..."
address,"{""address"": [""address_id"", ""address"", ""address...",32,"[-0.027424100786447525, 0.024846235290169716, ..."


,metadata_str,md_str_tokens,embedding
film,"table_name:film\n{""col_name"": ""film_id"", ""type...",443,"[-0.0006032844539731741, 0.024174265563488007,..."
language,"table_name:language\n{""col_name"": ""language_id...",103,"[0.0010160236852243543, 0.02792864479124546, 0..."
actor,"table_name:actor\n{""col_name"": ""actor_id"", ""ty...",138,"[-0.0034067181404680014, 0.005942848976701498,..."
store,"table_name:store\n{""col_name"": ""store_id"", ""ty...",139,"[-0.006352687254548073, 0.021349918097257614, ..."
address,"table_name:address\n{""col_name"": ""address_id"",...",271,"[-0.0030516614206135273, 0.02602650411427021, ..."


In [7]:
print('El numero de tablas que tiene la actual base de datos es: ')
print(len(simple_md_df), len(full_md_df))

El numero de tablas que tiene la actual base de datos es: 
15 15


## Extraemos los metadatos más similares a una consulta del usuario

La funcion search most similar metadata tiene dos limitantes el numero de tablas minimo que y el numero maximo de tokens 

In [11]:
consulta_pagila = '''
dame los nombres de los actores y actrices que hayan participado en 
peliculas con una duracion superior 180 minutos. incluye tambien la 
duración exacta de la pelicula
'''

consulta_rna_central_1 = "Muestra todas las secuencias de ARN que tienen una longitud mayor a 200 nucleótidos."
consulta_rna_central_2 =  "Encuentra las entradas de ARN que han sido identificadas en organismos específicos como 'Homo sapiens'."
consulta_rna_central_3 = "Lista los estudios que han citado secuencias de ARN particulares más de 50 veces."
consulta_rna_central_4 =  "Obtén los nombres de los investigadores que han contribuido con más de 10 secuencias de ARN a la base de datos."
consulta_rna_central_5 = "Muestra las secuencias de ARN que se han descubierto en el último año."

token_limit = 1000
n_tablas = 15

most_similar_md_simple = search_most_similar_md(prompt= consulta_pagila,metadata=simple_md_df, n_resultados=n_tablas, lim_tokens=token_limit)
most_similar_md_full = search_most_similar_md(prompt= consulta_pagila,metadata=full_md_df, n_resultados=n_tablas, lim_tokens=token_limit )

In [13]:
display(most_similar_md_simple)
display(most_similar_md_full)

,metadata_str,md_str_tokens,similarity,embedding,token_cumsum
film_actor,"{""film_actor"": [""actor_id"", ""film_id"", ""last_u...",16,0.733055,"[-0.033447034657001495, 0.003196529345586896, ...",16
film,"{""film"": [""film_id"", ""title"", ""description"", ""...",53,0.732425,"[-0.010325274430215359, 0.012428195215761662, ...",69
actor,"{""actor"": [""actor_id"", ""first_name"", ""last_nam...",19,0.725807,"[-0.03288605436682701, -0.005693415179848671, ...",88
film_category,"{""film_category"": [""film_id"", ""category_id"", ""...",16,0.700418,"[-0.025663767009973526, 0.014611182734370232, ...",104
inventory,"{""inventory"": [""inventory_id"", ""film_id"", ""sto...",19,0.692363,"[-0.01771996170282364, 0.002168742474168539, 0...",123
language,"{""language"": [""language_id"", ""name"", ""last_upd...",14,0.686968,"[-0.029717877507209778, 0.01876918599009514, 0...",137
rental,"{""rental"": [""rental_id"", ""rental_date"", ""inven...",34,0.684099,"[-0.025464307516813278, 0.0028804554603993893,...",171
staff,"{""staff"": [""staff_id"", ""first_name"", ""last_nam...",42,0.676507,"[-0.018970182165503502, 0.008695820346474648, ...",213
category,"{""category"": [""category_id"", ""name"", ""last_upd...",14,0.674761,"[-0.030182400718331337, 0.017262794077396393, ...",227
address,"{""address"": [""address_id"", ""address"", ""address...",32,0.674351,"[-0.027424100786447525, 0.024846235290169716, ...",259


,metadata_str,md_str_tokens,similarity,embedding,token_cumsum
actor,"table_name:actor\n{""col_name"": ""actor_id"", ""ty...",138,0.742399,"[-0.0034067181404680014, 0.005942848976701498,...",138
film_actor,"table_name:film_actor\n{""col_name"": ""actor_id""...",107,0.740795,"[-0.00541864987462759, 0.010073642246425152, 0...",245
film,"table_name:film\n{""col_name"": ""film_id"", ""type...",443,0.723773,"[-0.0006032844539731741, 0.024174265563488007,...",688
film_category,"table_name:film_category\n{""col_name"": ""film_i...",107,0.718325,"[0.0017659964505583048, 0.02148573473095894, -...",795
inventory,"table_name:inventory\n{""col_name"": ""inventory_...",138,0.709392,"[0.00015317850920837373, 0.015428561717271805,...",933


In [14]:
print(len(most_similar_md_simple))
print(len(most_similar_md_full))

15
5


In [15]:
from utils import get_string_from_metadata_df

simple_md_str = get_string_from_metadata_df(most_similar_md_simple)
full_md_str = get_string_from_metadata_df(most_similar_md_full)

print('SIMPLE METADATA')
print(simple_md_str)
print()
print('FULL METADATA')
print(full_md_str)

SIMPLE METADATA
{"film_actor": ["actor_id", "film_id", "last_update"]}
{"film": ["film_id", "title", "description", "release_year", "language_id", "rental_duration", "rental_rate", "length", "replacement_cost", "rating", "last_update", "special_features", "fulltext"]}
{"actor": ["actor_id", "first_name", "last_name", "last_update"]}
{"film_category": ["film_id", "category_id", "last_update"]}
{"inventory": ["inventory_id", "film_id", "store_id", "last_update"]}
{"language": ["language_id", "name", "last_update"]}
{"rental": ["rental_id", "rental_date", "inventory_id", "customer_id", "return_date", "staff_id", "last_update"]}
{"staff": ["staff_id", "first_name", "last_name", "address_id", "email", "store_id", "active", "username", "password", "last_update", "picture"]}
{"category": ["category_id", "name", "last_update"]}
{"address": ["address_id", "address", "address2", "district", "city_id", "postal_code", "phone", "last_update"]}
{"city": ["city_id", "city", "country_id", "last_update

## Todo el pipeline en una función

In [16]:
from utils import nlp_to_BBBD_with_metadata_to_text_df_2

consulta_nlp = input('dame tu consutla: ')
print(consulta_nlp)
print()

# consulta_nlp=  '''
# dame los nombres de los actores y actrices que hayan participado en 
# peliculas con una duracion superior 180 minutos. incluye tambien la 
# duración exacta de la pelicula
# '''
pagila = {
    'bbdd_name':'pagila', 
    'user'     :'postgres', 
    'password' :'123', 
    'host'     :'localhost', 
    'port'     :'5432'
}

rna_central = {
    'bbdd_name': 'pfmegrnargs', 
    'user'     : 'reader', 
    'password' : 'NWDMCE5xdipIjRrp', 
    'host'     : 'hh-pgsql-public.ebi.ac.uk', 
    'port'     : '5432'
}

token_limit= 2000
metadata_mode = 'simple'
n_tablas = 100

params = {
    'consulta_nlp' : consulta_nlp, 
    'metadata_mode': metadata_mode, 
    'n_tablas'     : n_tablas, 
    'max_tokens'   : token_limit
}

all_params_pagila = {**pagila, **params}
# all_params_rna = {**rna_central, **params}


tabla_texto= nlp_to_BBBD_with_metadata_to_text_df_2(
    **all_params_pagila
)

# tabla_texto= nlp_to_BBBD_with_metadata_to_text_df_2(
#     **all_params_rna
# )


print('\n')
print(tabla_texto)

dame los nombres de los actores y actrices que participen en peliculas de mas de 180 minutos y devuleve el titulo de la pelicula donde hayan trabajado con su duración exacta

El archivo ya existe: data/pagila_simple_metadata.pickle
El archivo ya existe: data/pagila_full_metadata.pickle
SELECT a.first_name, a.last_name, f.title, f.length
FROM actor a
JOIN film_actor fa ON a.actor_id = fa.actor_id
JOIN film f ON fa.film_id = f.film_id
WHERE f.length > 180; 



|     | first_name   | last_name    | title              |   length |
|----:|:-------------|:-------------|:-------------------|---------:|
|   0 | Penelope     | Guiness      | King Evolution     |      184 |
|   1 | Ed           | Chase        | Young Language     |      183 |
|   2 | Jennifer     | Davis        | Reds Pocus         |      182 |
|   3 | Johnny       | Lollobrigida | Frontier Cabin     |      183 |
|   4 | Johnny       | Lollobrigida | Love Suicides      |      181 |
|   5 | Johnny       | Lollobrigida | Soldiers 